In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import typing
import imageio
import numpy as np
import tensorflow as tf

In [ ]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
seed = 6969
# seed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)

Random Seed:  6969


In [ ]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

FileNotFoundError: Couldn't find any class folder in /kaggle/input/.

In [7]:
!unzip NM.zip

Archive:  NM.zip
   creating: New folder (2)/New folder/
  inflating: New folder (2)/New folder/0_2000.jpg  
  inflating: New folder (2)/New folder/1_2000.jpg  
  inflating: New folder (2)/New folder/10_2000.jpg  
  inflating: New folder (2)/New folder/11_2000.jpg  
  inflating: New folder (2)/New folder/12_2000.jpg  
  inflating: New folder (2)/New folder/13_2000.jpg  
  inflating: New folder (2)/New folder/14_2000.jpg  
  inflating: New folder (2)/New folder/15_2000.jpg  
  inflating: New folder (2)/New folder/16_2000.jpg  
  inflating: New folder (2)/New folder/17_2000.jpg  
  inflating: New folder (2)/New folder/18_2000.jpg  
  inflating: New folder (2)/New folder/19_2000.jpg  
  inflating: New folder (2)/New folder/2_2000.jpg  
  inflating: New folder (2)/New folder/20_2000.jpg  
  inflating: New folder (2)/New folder/3_2000.jpg  
  inflating: New folder (2)/New folder/4_2000.jpg  
  inflating: New folder (2)/New folder/5_2000.jpg  
  inflating: New folder (2)/New folder/6_2000.jp

In [8]:
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator



from keras import mixed_precision


In [10]:
# celebA dataset path
dataset_path = "/content/New folder (2)"

# Set the input shape and size for the generator and discriminator
batch_size = 128
img_shape = (64, 64, 3) # The shape of the input image, input to the discriminator
noise_dim = 128 # The dimension of the noise vector, input to the generator
model_path = '/content/New folder (2)/New folder'
os.makedirs(model_path, exist_ok=True)

# Define your data generator
datagen = ImageDataGenerator(
    preprocessing_function=lambda x: (x / 127.5) - 1.0,  # Normalize image pixel values to [-1, 1]
    horizontal_flip=True  # Data augmentation
)

# Create a generator that yields batches of images
train_generator = datagen.flow_from_directory(
    directory=dataset_path,  # Path to directory containing images
    target_size=img_shape[:2],  # Size of images (height, width)
    batch_size=batch_size,
    class_mode=None,  # Do not use labels
    shuffle=True,  # Shuffle the data
)

Found 21 images belonging to 1 classes.


In [24]:
print(train_generator[0])

[[[[-0.9529412  -0.6313726  -0.2862745 ]
   [-1.         -0.69411767 -0.3490196 ]
   [-1.         -0.7490196  -0.41960782]
   ...
   [-0.88235295 -0.44313723  0.03529418]
   [-0.90588236 -0.46666664 -0.00392157]
   [-0.9529412  -0.49019605 -0.03529412]]

  [[-0.8980392  -0.5764706  -0.21568626]
   [-0.90588236 -0.58431375 -0.2235294 ]
   [-0.96862745 -0.64705884 -0.30196077]
   ...
   [-0.8666667  -0.40392154  0.06666672]
   [-0.88235295 -0.41960782  0.03529418]
   [-0.8980392  -0.4352941   0.0196079 ]]

  [[-0.9137255  -0.5921569  -0.18431371]
   [-0.9607843  -0.6313726  -0.23921567]
   [-1.         -0.69411767 -0.3098039 ]
   ...
   [-0.8901961  -0.42745095  0.02745104]
   [-0.8980392  -0.41960782  0.02745104]
   [-0.8980392  -0.42745095  0.04313731]]

  ...

  [[-0.27843136 -0.5529412  -0.3960784 ]
   [ 0.3176471   0.05882359  0.19215691]
   [ 0.8039216   0.58431375  0.69411767]
   ...
   [ 0.5137255   0.41176474  0.13725495]
   [-0.16862744 -0.23921567 -0.46666664]
   [ 0.73333335 

In [11]:
import tensorflow as tf
from keras import layers
latent_dim=128
# Define the generator model
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)

generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8192)              1056768   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 16, 16, 128)       262272    
 anspose)                                                        
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 32, 32, 256)       524544    
 Transpose)                                                      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 256)       0 

In [12]:
import tensorflow as tf
from keras import layers

discriminator= keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="linear"),
    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)            

In [13]:
def gradient_penalty(
        self,
        real_samples: tf.Tensor,
        fake_samples: tf.Tensor,
        discriminator: tf.keras.models.Model
    ) -> tf.Tensor:
    """ Calculates the gradient penalty.

    Gradient penalty is calculated on an interpolated data
    and added to the discriminator loss.
    """
    batch_size = tf.shape(real_samples)[0]

    # Generate random values for epsilon
    epsilon = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0, maxval=1)

    # 1. Interpolate between real and fake samples
    interpolated_samples = epsilon * real_samples + ((1 - epsilon) * fake_samples)

    with tf.GradientTape() as tape:
        tape.watch(interpolated_samples)
        # 2. Get the discriminator's output for the interpolated image
        logits = discriminator(interpolated_samples, training=True)

    # 3. Calculate the gradients w.r.t to the interpolated image
    gradients = tape.gradient(logits, interpolated_samples)

    # 4. Calculate the L2 norm of the gradients.
    gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))

    # 5. Calculate gradient penalty
    gradient_penalty = tf.reduce_mean((gradients_norm - 1.0) ** 2)

    return gradient_penalty

In [14]:
class WGAN_GP(tf.keras.models.Model):
    def __init__(
            self,
            discriminator: tf.keras.models.Model,
            generator: tf.keras.models.Model,
            noise_dim: int,
            discriminator_extra_steps: int=5,
            gp_weight: typing.Union[float, int]=10.0
        ) -> None:
        super(WGAN_GP, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.noise_dim = noise_dim
        self.discriminator_extra_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(
            self,
            discriminator_opt: tf.keras.optimizers.Optimizer,
            generator_opt: tf.keras.optimizers.Optimizer,
            discriminator_loss: typing.Callable,
            generator_loss: typing.Callable,
            **kwargs
        ) -> None:
        super(WGAN_GP, self).compile(**kwargs)
        self.discriminator_opt = discriminator_opt
        self.generator_opt = generator_opt
        self.discriminator_loss = discriminator_loss
        self.generator_loss = generator_loss

    def add_instance_noise(self, x: tf.Tensor, stddev: float=0.1) -> tf.Tensor:
        """ Adds instance noise to the input tensor."""
        noise = tf.random.normal(tf.shape(x), mean=0.0, stddev=stddev, dtype=x.dtype)
        return x + noise

    def gradient_penalty(
            self,
            real_samples: tf.Tensor,
            fake_samples: tf.Tensor,
            discriminator: tf.keras.models.Model
        ) -> tf.Tensor:
        """ Calculates the gradient penalty.

        Gradient penalty is calculated on an interpolated data
        and added to the discriminator loss.
        """
        batch_size = tf.shape(real_samples)[0]

        # Generate random values for epsilon
        epsilon = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0, maxval=1)

        # 1. Interpolate between real and fake samples
        interpolated_samples = epsilon * real_samples + ((1 - epsilon) * fake_samples)

        with tf.GradientTape() as tape:
            tape.watch(interpolated_samples)
            # 2. Get the discriminator's output for the interpolated image
            logits = discriminator(interpolated_samples, training=True)

        # 3. Calculate the gradients w.r.t to the interpolated image
        gradients = tape.gradient(logits, interpolated_samples)

        # 4. Calculate the L2 norm of the gradients.
        gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))

        # 5. Calculate gradient penalty
        gradient_penalty = tf.reduce_mean((gradients_norm - 1.0) ** 2)

        return gradient_penalty

    def train_step(self, real_samples: tf.Tensor) -> typing.Dict[str, float]:
        batch_size = tf.shape(real_samples)[0]
        noise = tf.random.normal([batch_size, self.noise_dim])
        gps = []

        # Step 1. Train the discriminator with both real and fake samples
        # Train the discriminator more often than the generator
        for _ in range(self.discriminator_extra_steps):

            # Step 1. Train the discriminator with both real images and fake images
            with tf.GradientTape() as tape:
                fake_samples = self.generator(noise, training=True)
                pred_real = self.discriminator(real_samples, training=True)
                pred_fake = self.discriminator(fake_samples, training=True)

                # Add instance noise to real and fake samples
                real_samples = self.add_instance_noise(real_samples)
                fake_samples = self.add_instance_noise(fake_samples)

                # Calculate the WGAN-GP gradient penalty
                gp = self.gradient_penalty(real_samples, fake_samples, self.discriminator)
                gps.append(gp)

                # Add gradient penalty to the original discriminator loss
                disc_loss = self.discriminator_loss(pred_real, pred_fake) + gp * self.gp_weight

            # Compute discriminator gradients
            grads = tape.gradient(disc_loss, self.discriminator.trainable_variables)

            # Update discriminator weights
            self.discriminator_opt.apply_gradients(zip(grads, self.discriminator.trainable_variables))

        # Step 2. Train the generator
        with tf.GradientTape() as tape:
            fake_samples = self.generator(noise, training=True)
            pred_fake = self.discriminator(fake_samples, training=True)
            gen_loss = self.generator_loss(pred_fake)

        # Compute generator gradients
        grads = tape.gradient(gen_loss, self.generator.trainable_variables)

        # Update generator wieghts
        self.generator_opt.apply_gradients(zip(grads, self.generator.trainable_variables))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(real_samples, fake_samples)

        results = {m.name: m.result() for m in self.metrics}
        results.update({"d_loss": disc_loss, "g_loss": gen_loss, "gp": tf.reduce_mean(gps)})

        return results

In [15]:
class ResultsCallback(tf.keras.callbacks.Callback):
    """ Callback for generating and saving images during training."""
    def __init__(
            self,
            noise_dim: int,
            output_path: str,
            examples_to_generate: int=16,
            grid_size: tuple=(4, 4),
            spacing: int=5,
            gif_size: tuple=(416, 416),
            duration: float=0.1,
            save_model: bool=True
        ) -> None:
        super(ResultsCallback, self).__init__()
        self.seed = tf.random.normal([examples_to_generate, noise_dim])
        self.results = []
        self.output_path = output_path
        self.results_path = output_path + '/results'
        self.grid_size = grid_size
        self.spacing = spacing
        self.gif_size = gif_size
        self.duration = duration
        self.save_model = save_model

        os.makedirs(self.results_path, exist_ok=True)

    def save_plt(self, epoch: int, results: np.ndarray):
        # construct an image from generated images with spacing between them using numpy
        w, h , c = results[0].shape
        # construct grind with self.grid_size
        grid = np.zeros((self.grid_size[0] * w + (self.grid_size[0] - 1) * self.spacing, self.grid_size[1] * h + (self.grid_size[1] - 1) * self.spacing, c), dtype=np.uint8)
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                grid[i * (w + self.spacing):i * (w + self.spacing) + w, j * (h + self.spacing):j * (h + self.spacing) + h] = results[i * self.grid_size[1] + j]

        grid = cv2.cvtColor(grid, cv2.COLOR_RGB2BGR)

        # save the image
        cv2.imwrite(f'{self.results_path}/img_{epoch}.png', grid)

        # save image to memory resized to gif size
        self.results.append(cv2.resize(grid, self.gif_size, interpolation=cv2.INTER_AREA))

    def on_epoch_end(self, epoch: int, logs: dict=None):
        # Define your custom code here that should be executed at the end of each epoch
        predictions = self.model.generator(self.seed, training=False)
        predictions_uint8 = (predictions * 127.5 + 127.5).numpy().astype(np.uint8)
        self.save_plt(epoch, predictions_uint8)

        if self.save_model:
            # save keras model to disk
            models_path = os.path.join(self.output_path, "model")
            os.makedirs(models_path, exist_ok=True)
            self.model.discriminator.save(models_path + "/discriminator.h5")
            self.model.generator.save(models_path + "/generator.h5")

    def on_train_end(self, logs: dict=None):
        # save the results as a gif with imageio

        # Create a list of imageio image objects from the OpenCV images
        # image is in BGR format, convert to RGB format when loading
        imageio_images = [imageio.core.util.Image(image[...,::-1]) for image in self.results]

        # Write the imageio images to a GIF file
        imageio.mimsave(self.results_path + "/output.gif", imageio_images, duration=self.duration)

In [17]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save("/content/output.png")

In [18]:

class LRSheduler(tf.keras.callbacks.Callback):
    """Learning rate scheduler for WGAN-GP"""
    def __init__(self, decay_epochs: int, tb_callback=None, min_lr: float=0.00001):
        super(LRSheduler, self).__init__()
        self.decay_epochs = decay_epochs
        self.min_lr = min_lr
        self.tb_callback = tb_callback
        self.compiled = False

    def on_epoch_end(self, epoch, logs=None):
        if not self.compiled:
            self.generator_lr = self.model.generator_opt.lr.numpy()
            self.discriminator_lr = self.model.discriminator_opt.lr.numpy()
            self.compiled = True

        if epoch < self.decay_epochs:
            new_g_lr = max(self.generator_lr * (1 - (epoch / self.decay_epochs)), self.min_lr)
            self.model.generator_opt.lr.assign(new_g_lr)
            new_d_lr = max(self.discriminator_lr * (1 - (epoch / self.decay_epochs)), self.min_lr)
            self.model.discriminator_opt.lr.assign(new_d_lr)
            print(f"Learning rate generator: {new_g_lr}, discriminator: {new_d_lr}")

            # Log the learning rate on TensorBoard
            if self.tb_callback is not None:
                writer = self.tb_callback._writers.get('train')  # get the writer from the TensorBoard callback
                with writer.as_default():
                    tf.summary.scalar('generator_lr', data=new_g_lr, step=epoch)
                    tf.summary.scalar('discriminator_lr', data=new_d_lr, step=epoch)
                    writer.flush()

In [19]:
# Wasserstein loss for the discriminator
def discriminator_w_loss(pred_real, pred_fake):
    real_loss = tf.reduce_mean(pred_real)
    fake_loss = tf.reduce_mean(pred_fake)
    return fake_loss - real_loss

# Wasserstein loss for the generator
def generator_w_loss(pred_fake):
    return -tf.reduce_mean(pred_fake)

In [20]:

generator_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5, beta_2=0.9)


In [25]:
callback = ResultsCallback(noise_dim=noise_dim, output_path=model_path, duration=0.04)
tb_callback = TensorBoard(model_path + '/logs')
lr_scheduler = LRSheduler(decay_epochs=500, tb_callback=tb_callback)

gan = WGAN_GP(discriminator=discriminator, generator=generator, noise_dim=noise_dim, discriminator_extra_steps=5)
gan.compile(discriminator_optimizer, generator_optimizer, discriminator_w_loss, generator_w_loss, run_eagerly=False,metrics=["accuracy"])

history=gan.fit(train_generator, epochs=1000, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)])

Epoch 1/1000
1/1 [==============================] - 6s 6s/step - accuracy: 0.5630 - d_loss: -95.4059 - g_loss: -12.5911 - gp: 4.3391
Epoch 2/1000
1/1 [==============================] - 0s 452ms/step - accuracy: 0.5497 - d_loss: -91.8594 - g_loss: -3.8041 - gp: 4.4931
Epoch 3/1000
1/1 [==============================] - 0s 458ms/step - accuracy: 0.5526 - d_loss: -94.6962 - g_loss: -5.5907 - gp: 4.3626
Epoch 4/1000
1/1 [==============================] - 0s 455ms/step - accuracy: 0.5656 - d_loss: -97.9303 - g_loss: -5.8415 - gp: 4.4760
Epoch 5/1000
1/1 [==============================] - 0s 472ms/step - accuracy: 0.5479 - d_loss: -94.2705 - g_loss: 17.6360 - gp: 4.8378
Epoch 6/1000
1/1 [==============================] - 0s 446ms/step - accuracy: 0.5465 - d_loss: -92.3698 - g_loss: 19.5342 - gp: 4.4383
Epoch 7/1000
1/1 [==============================] - 0s 447ms/step - accuracy: 0.5509 - d_loss: -98.9331 - g_loss: 0.7068 - gp: 4.7338
Epoch 8/1000
1/1 [==============================] - 0s 443